In [1]:
import pandas as pd

loans_file = r'LoanData.csv'

used_cols = ['UserName','NewCreditCustomer','LoanApplicationStartedDate','ApplicationSignedHour',
    'ApplicationSignedWeekday','VerificationType','LanguageCode','Age','Gender','Country',
    'AppliedAmount','Interest','LoanDuration','County','City','Education','EmploymentDurationCurrentEmployer',
    'HomeOwnershipType','IncomeTotal','MonthlyPaymentDay','ModelVersion','ExpectedLoss',
    'Rating_V2','LossGivenDefault','ProbabilityOfDefault','LiabilitiesTotal']

used_cols.append('LoanApplicationStartedDate')
used_cols.append('DefaultDate')


df = pd.read_csv(loans_file,
                 usecols = used_cols,
                 low_memory=False,
                 parse_dates=['LoanApplicationStartedDate','DefaultDate'],
                 converters={'County': str, 'City': str, 'EmploymentDurationCurrentEmployer': str, 
                 'Rating_V2' : str},
                 #, 'CreditScoreEsMicroL': str,'CreditScoreEsEquifaxRisk': str,'CreditScoreFiAsiakasTietoRiskGrade': str},
                 compression='infer',
                 infer_datetime_format=False)

df = df[(df['LoanApplicationStartedDate'] < '2016-06-01')]
df['DebtToIncome'] = df['LiabilitiesTotal']/df['IncomeTotal']
df['Status'] = df['DefaultDate'].isnull()

In [2]:
df.tail(4)

,UserName,NewCreditCustomer,LoanApplicationStartedDate,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,Gender,Country,...,LiabilitiesTotal,MonthlyPaymentDay,ExpectedLoss,LossGivenDefault,ProbabilityOfDefault,DefaultDate,ModelVersion,Rating_V2,DebtToIncome,Status
36642,BO542A57A,False,2016-02-17 09:16:05,11,5,4.0,4,37,0.0,FI,...,2488.09,23,0.117339,0.68,0.170865,NaT,2.0,D,1.145529,True
36643,BO65A965,False,2016-02-04 15:41:12,14,4,1.0,6,37,0.0,ES,...,1877.00,1,0.503420,0.75,0.573979,NaT,2.0,HR,1.210968,True
36644,BO9577433,True,2015-08-12 12:29:10,20,2,1.0,3,58,1.0,EE,...,359.00,10,0.065220,0.65,0.077184,NaT,1.0,D,1.025714,True
36645,avo1007,False,2015-06-13 09:01:56,10,2,4.0,1,64,0.0,EE,...,804.20,27,0.025470,0.65,0.030142,2016-07-13,1.0,AA,0.927566,False


In [3]:
df.describe()

,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,Gender,AppliedAmount,Interest,LoanDuration,Education,HomeOwnershipType,IncomeTotal,LiabilitiesTotal,MonthlyPaymentDay,ExpectedLoss,LossGivenDefault,ProbabilityOfDefault,ModelVersion,DebtToIncome
count,24609.000000,24609.000000,24556.000000,24609.000000,24609.000000,24556.000000,24609.000000,24609.000000,24609.000000,24556.000000,22919.000000,24609.000000,24609.000000,24609.000000,21883.000000,21883.000000,21883.000000,21883.000000,2.457500e+04
mean,14.048397,3.938722,2.904300,2.772238,37.844528,0.451417,2648.712933,31.140816,41.651225,3.788524,3.324796,2142.425276,752.729998,12.003698,0.190155,0.738205,0.207205,0.991546,inf
std,4.366943,1.571750,1.184525,2.124789,11.518482,0.515334,2361.785953,12.219761,20.104406,0.996569,2.478903,4125.723798,1448.846802,7.395431,0.171552,0.126672,0.169876,0.639877,NaN
min,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,31.955800,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,11.000000,3.000000,2.000000,1.000000,29.000000,0.000000,1000.000000,24.460000,24.000000,3.000000,1.000000,790.000000,350.000000,6.000000,0.074237,0.650000,0.083964,1.000000,3.428044e-01
50%,14.000000,4.000000,3.000000,1.000000,36.000000,0.000000,2000.000000,29.670000,48.000000,4.000000,3.000000,1193.000000,559.000000,10.000000,0.125474,0.650000,0.137053,1.000000,5.016807e-01
75%,17.000000,5.000000,4.000000,4.000000,46.000000,1.000000,3190.000000,34.000000,60.000000,5.000000,4.000000,1900.000000,932.000000,15.000000,0.246811,0.900000,0.312118,1.000000,6.800000e-01
max,23.000000,7.000000,4.000000,22.000000,77.000000,2.000000,10630.000000,93.650000,60.000000,5.000000,9.000000,228550.000000,172510.000000,28.000000,1.008540,0.900000,0.879805,2.000000,inf


In [88]:
import matplotlib.pyplot as plt
import numpy as np

hours = pd.pivot_table(df, values='Age', index=['ApplicationSignedHour'], columns=['Status'], aggfunc=np.size)
hours.head()

Status,False,True
ApplicationSignedHour,,
0,137,158
1,79,83
2,47,49
3,24,20
4,32,24


In [90]:
#hours['Status']
#plt.bar(list(hours['False']), hours.values)
#plt.bar(list(hours.index.values), hours.values, bottom=hours.values)
#plt.show()

KeyError: 'Status'